In [4]:
from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import MessageToJson
import urllib.request
import time
import pytz
import json
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import date

def scheduleDateTime(stop_time):
    dt=date.today()
    time_obj = datetime.strptime(stop_time, "%H:%M:%S").time()
    dt_with_time = datetime.combine(dt, time_obj)
    dt_with_time = pytz.timezone('Europe/Warsaw').localize(dt_with_time)
    return dt_with_time

feed = gtfs_realtime_pb2.FeedMessage()
response = urllib.request.urlopen('https://mkuran.pl/gtfs/wkd.pb')
feed.ParseFromString(response.read())

rozklad = pd.read_csv(r'/home/humback/Pulpit/wkd/wkd_gtfs/stop_times.txt', sep=',')
print(rozklad)
json_wkd = MessageToJson(feed)
#print(json_wkd)
json_wkd=json.loads(json_wkd)
json_wkd=json_wkd['entity']
#print(json_wkd)



ImportError: cannot import name 'builder' from 'google.protobuf.internal' (/usr/lib/python3/dist-packages/google/protobuf/internal/__init__.py)

opis

In [ ]:
df = pd.json_normalize(json_wkd)


In [ ]:
df = pd.json_normalize(json_wkd, record_path=['tripUpdate','stopTimeUpdate'],meta=['id'])
tz = pytz.timezone('Europe/Warsaw')
df['departure.time'] = pd.to_datetime(df['departure.time'], unit='s').apply(lambda x: pd.Timestamp(x).tz_localize(None).tz_localize('UTC').tz_convert(tz))
print(df)

In [ ]:
actual = df[df['stopId'] == 'nwwar']
  
print(actual)

rozklad_filtr = rozklad[rozklad['stop_id'] == 'nwwar']
  
print(rozklad_filtr)

In [ ]:
# merge dataframes based on key_col1 and key_col2
merged_df = pd.merge(actual, rozklad, left_on=['id','stopId'], right_on=['trip_id','stop_id'], how='left')

merged_df = merged_df.drop(columns=['trip_id','stop_id','departure_time'])
merged_df = merged_df.sort_values(by='departure.time')

merged_df['arrival_time'] = merged_df['arrival_time'].fillna(merged_df['departure.time'].apply(lambda x: x.strftime('%H:%M:%S')))
#print(merged_df)

merged_df['arrival_time'] = merged_df['arrival_time'].apply(scheduleDateTime)

merged_df['delay']=merged_df['departure.time']-merged_df['arrival_time']

merged_df['delay'] = merged_df['delay'].apply(lambda x: x.total_seconds() / 60)


merged_df['direction'] = merged_df['id'].apply(lambda x: 'W' if int(x[2:]) % 2 == 0 else 'P')
merged_df=merged_df[['direction','id','stopId','delay','arrival_time','departure.time']]
mask = merged_df['direction'] == 'P'
print(merged_df[mask].to_string(index=False))

mask = merged_df['direction'] == 'W'
print(merged_df[mask].to_string(index=False))